In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class RegNet(nn.Module):
    def __init__(self):
        super(RegNet, self).__init__()
        self.fc1 = nn.Linear(1000, 5000)
        self.fc2 = nn.Linear(5000, 10000)
        self.fc3 = nn.Linear(10000,3000)
        self.fc4 = nn.Linear(3000,100)
        self.fc5 = nn.Linear(100,1)
        self.thr = nn.ReLU()

    def forward(self,x):
        x=self.thr(self.fc1(x))
        x=self.thr(self.fc2(x))
        x=self.thr(self.fc3(x))
        x=self.thr(self.fc4(x))
        x=self.fc5(x)
        return x 

import hdf5storage
mat = hdf5storage.loadmat('Heatwdata.mat') 
import torch
import h5py
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pylab as plt
import pickle
import scipy.io as sio
dtype = torch.double
device = torch.device("cpu")


H_right=mat['source']
H_wrong=mat['wsource']

#Convert into torch arrays
Hr=torch.from_numpy(H_right)
Hw=torch.from_numpy(H_wrong)

nValidation=100
nTrain=Hr.shape[1]-nValidation

print(nTrain)

# initial scrambling 
perm = torch.randperm(nTrain+nValidation)
Hr = Hr[:,perm]
Hw = Hw[:,perm]

# validation dataset
val_r = Hr[:,-nValidation:]
val_w = Hw[:,-nValidation:]


N=50
nEpochs=100

net = RegNet()
net = net.float()

# Loss and optimizer
learning_rate = 1e-1
criterion = nn.L1Loss() 
optimizer = torch.optim.Adagrad(net.parameters(), lr=learning_rate)


for t in range(nEpochs):
    tloss=0
    val_loss = 0
    perm = torch.randperm(nTrain)
    for b_ix in np.arange(0,nTrain,N):
        
        
        ##Pre-processing sparse codes for input 
        xr=Hr[:,perm[b_ix:b_ix+N]].reshape(N,Hr.shape[0])
        xw=Hw[:,perm[b_ix:b_ix+N]].reshape(N,Hw.shape[0])
        optimizer.zero_grad()
    
        normr = net(xr.float())
        normw = net(xw.float())

        loss1 = -criterion(normr,normw) 
        
        
        # Backward and optimize
        
        loss1.backward()
        optimizer.step()

        with torch.no_grad():
            tloss+=loss1.item()
            if b_ix % nTrain == 0:
                vnormr=net(val_r.reshape(val_r.shape[1],val_r.shape[0]).float())
                vnormw=net(val_w.reshape(val_w.shape[1],val_w.shape[0]).float())
                vloss = -criterion(vnormr,vnormw).float()
                print('Epoch:{:d}| T Loss:{:.10f} | V Loss:{:.10f}'.format(t+1, tloss/nTrain, vloss/nValidation))
                tloss=0
                vloss=0


900
Epoch:1| T Loss:-0.0000590547 | V Loss:-38532.9492187500
Epoch:2| T Loss:-84004666955.0933380127 | V Loss:-836213604352.0000000000
Epoch:3| T Loss:-1277510432995.5556640625 | V Loss:-9196233818112.0000000000
Epoch:4| T Loss:-5352196760316.5869140625 | V Loss:-33597503307776.0000000000
Epoch:5| T Loss:-11863755517678.9335937500 | V Loss:-80569870319616.0000000000
Epoch:6| T Loss:-24107004801260.6562500000 | V Loss:-144765882466304.0000000000
Epoch:7| T Loss:-37789229190275.9843750000 | V Loss:-243419754528768.0000000000
Epoch:8| T Loss:-56840255756916.0546875000 | V Loss:-368144396845056.0000000000
Epoch:9| T Loss:-81043140930874.0312500000 | V Loss:-522637289717760.0000000000
Epoch:10| T Loss:-106128403274865.7812500000 | V Loss:-704956000632832.0000000000
Epoch:11| T Loss:-163581865829189.4062500000 | V Loss:-911335286112256.0000000000
Epoch:12| T Loss:-218870273547086.5000000000 | V Loss:-1193128761491456.0000000000
Epoch:13| T Loss:-278724250589375.1562500000 | V Loss:-150578734

Epoch:98| T Loss:-66219109106943848.0000000000 | V Loss:-331962763951734784.0000000000
Epoch:99| T Loss:-55944656730493344.0000000000 | V Loss:-340878428863463424.0000000000
Epoch:100| T Loss:-53457238905053224.0000000000 | V Loss:-349431598535409664.0000000000
